In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session*

In [ ]:
train=pd.read_csv("/kaggle/input/meesho-ocr/df1_with_ocr.csv")
test=pd.read_csv("/kaggle/input/meesho-ocr/df2_with_ocr.csv")

In [ ]:
from IPython.display import display

In [ ]:
value_counts = train["ocr_text"].value_counts()

# Convert to a DataFrame
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['value', 'frequency']

# Calculate cumulative frequency
value_counts_df['cumulative_frequency'] = value_counts_df['frequency'].cumsum()

# Show the result
value_counts_df.head(30)


In [ ]:
df = train 
ocr_text_filtered = {}
replacement = {}

# Process each category separately
for cat in df.Category.unique():
    # Filter ocr_text for the current category where it has more than 3 characters and count occurrences
    ocr_text_counts = df[(df.Category == cat) & (df.ocr_text.str.len() >= 3)]['ocr_text'].value_counts()
    
    # Keep only values that appear more than 10 times in the current category
    filtered_values = ocr_text_counts[ocr_text_counts >= 44].index.to_numpy()
    ocr_text_filtered[cat] = filtered_values
    
    # Filter the DataFrame for rows that match the filtered ocr_text values for this category
    cat_filtered_df = df[(df.Category == cat) & (df['ocr_text'].isin(filtered_values))]
    
    # Define a function to get the most common non-null value, or null if all values are null
    def most_common_or_null(series):
        non_null_series = series.dropna()
        value_counts = non_null_series.value_counts()
        common_values = value_counts[value_counts >= 3]
        
        if not common_values.empty:
            return common_values.idxmax()  # Most common value with more than 8 occurrences
        else:
            return None  # Return None if no value has more than 8 occurrences

    # Group by 'ocr_text' and apply the most_common_or_null function to each attribute column
    result_df = cat_filtered_df.groupby('ocr_text').agg({f'attr_{i}': most_common_or_null for i in range(1, 11)}).reset_index()
    
    # Initialize a dictionary for this category in replacement
    replacement[cat] = {}
    
    # Populate replacement dictionary with values from result_df for the current category
    for ind, row in result_df.iterrows():
        replacement[cat][row['ocr_text']] = [row[f'attr_{i}'] for i in range(1, 11)]

# replacement now has the structure replacement[cat][ocr_text]
replacement


In [ ]:
labels=[f"attr_{i}" for i in range (1,11)]

In [ ]:
cnt = 0

for ind, row in train.iterrows():
    # Skip rows where 'ocr_text' is NaN or does not exist in replacement for the given category
    if pd.isna(row.ocr_text) or row.ocr_text not in replacement[row.Category]:
        continue

    # Iterate through each label and update if needed
    for j, i in enumerate(labels):
        replacement_value = replacement[row.Category][row.ocr_text][j]
        # Check if replacement value is not NaN and differs from the current value
        if pd.notna(replacement_value) and replacement_value != train.loc[ind, i]:
            cnt += 1
            train.loc[ind, i] = replacement_value
cnt


In [ ]:
train.to_csv("pre_processed_train.csv")